### Import data from colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install Detectron2

In [2]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.


In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


### Important Imports

In [4]:
import logging
import os
from collections import OrderedDict

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from detectron2.data import transforms as T
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results, inference_on_dataset
)
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_train_loader, build_detection_test_loader, DatasetMapper

### Config setup and defining hyper-parameters

In [5]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)


def setup():
    global cfg
    """
    Create configs and perform basic setups.
    """
    print("START SETUP")
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")) # model config
    cfg.DATASETS.TRAIN = ("my_dataset_train",)  # defining name in dataloader training set
    cfg.DATASETS.TEST = ("my_dataset_val",)
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS=False
    cfg.INPUT.RANDOM_FLIP = 'none'

    cfg.DATALOADER.NUM_WORKERS = 8  # number of cpu cores for dataloader
#     cfg.MODEL.WEIGHTS = 'model_final.pth'
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml") # model wieght (alt : pre-trained weight )
    cfg.SOLVER.IMS_PER_BATCH = 8
    cfg.SOLVER.BASE_LR = 0.001 # initial LR

    cfg.SOLVER.WARMUP_ITERS = 500
    cfg.SOLVER.MAX_ITER = 2000 #adjust up if val mAP is still rising, adjust down if overfit
    cfg.SOLVER.STEPS = (750, 1500) # iters for LR decay
    cfg.SOLVER.GAMMA = 0.1  # decay factor for LR

    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 #number of classes
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

    cfg.SOLVER.CHECKPOINT_PERIOD = 500
    cfg.TEST.EVAL_PERIOD = 500
    # cfg.merge_from_list(args.opts)
    # cfg.freeze()
    default_setup(cfg, ({},))
    print("END SETUP")
    return cfg

### Register dataset for detectron2 to load it and Start training

In [6]:
def main():
    global cfg, trainer
    cfg = setup()
    register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/pdots/train.json", "/content/drive/MyDrive/pdots/final_dataset/train")
    register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/pdots/valid.json", "/content/drive/MyDrive/pdots/final_dataset/val")
    register_coco_instances("my_dataset_test", {}, "/content/drive/MyDrive/pdots/valid.json", "/content/drive/MyDrive/pdots/final_dataset/val")
    trainer = Trainer(cfg)
    trainer.resume_or_load(resume=False) # to restart training from saved checkpoint
    trainer.train()

    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    cfg.DATASETS.TEST = ("my_dataset_test", )
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set the testing threshold for this model
    predictor = DefaultPredictor(cfg)
    test_metadata = MetadataCatalog.get("my_dataset_test")

    return

if __name__ == "__main__":
    launch(main, num_gpus_per_machine = 1, dist_url="auto")

START SETUP
[09/15 10:10:00 detectron2]: Rank of current process: 0. World size: 1
[09/15 10:10:00 detectron2]: Rank of current process: 0. World size: 1
[09/15 10:10:00 detectron2]: Environment info:
-------------------------------  -----------------------------------------------------------------
sys.platform                     linux
Python                           3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
numpy                            1.23.5
detectron2                       0.6 @/content/detectron2/detectron2
detectron2._C                    not built correctly: No module named 'detectron2._C'
Compiler ($CXX)                  c++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
CUDA compiler                    Build cuda_11.8.r11.8/compiler.31833905_0
DETECTRON2_ENV_MODULE            <not set>
PyTorch                          2.0.1+cu118 @/usr/local/lib/python3.10/dist-packages/torch
PyTorch debug build              False
torch._C._GLIBCXX_USE_CXX11_ABI  False
GPU available      

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
model_final_b275ba.pkl: 167MB [00:01, 119MB/s]                           

[09/15 10:10:10 fvcore.common.checkpoint]: [Checkpointer] Loading from /root/.torch/iopath_cache/detectron2/COCO-Detection/faster_rcnn_R_50_FPN_1x/137257794/model_final_b275ba.pkl ...


[09/15 10:10:10 fvcore.common.checkpoint]: Reading a file from 'Detectron2 Model Zoo'
WARNING [09/15 10:10:11 fvcore.common.checkpoint]: Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
WARNING [09/15 10:10:11 fvcore.common.checkpoint]: Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
WARNING [09/15 10:10:11 fvcore.common.checkpoint]: Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
WARNING [09/15 10:10:11 fvcore.common.checkpoint]: Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the mo

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


...

[09/15 10:51:01 d2.evaluation.evaluator]: Inference done 11/56. Dataloading: 0.0887 s/iter. Inference: 0.1421 s/iter. Eval: 0.0005 s/iter. Total: 0.2312 s/iter. ETA=0:00:10
[09/15 10:51:01 d2.evaluation.evaluator]: Inference done 11/56. Dataloading: 0.0887 s/iter. Inference: 0.1421 s/iter. Eval: 0.0005 s/iter. Total: 0.2312 s/iter. ETA=0:00:10
[09/15 10:51:06 d2.evaluation.evaluator]: Inference done 31/56. Dataloading: 0.0322 s/iter. Inference: 0.2121 s/iter. Eval: 0.0016 s/iter. Total: 0.2465 s/iter. ETA=0:00:06
[09/15 10:51:06 d2.evaluation.evaluator]: Inference done 31/56. Dataloading: 0.0322 s/iter. Inference: 0.2121 s/iter. Eval: 0.0016 s/iter. Total: 0.2465 s/iter. ETA=0:00:06
[09/15 10:51:11 d2.evaluation.evaluator]: Total inference time: 0:00:11.182518 (0.219265 s / iter per device, on 1 devices)
[09/15 10:51:11 d2.evaluation.evaluator]: Total inference time: 0:00:11.182518 (0.219265 s / iter per device, on 1 devices)
[09/15 10:51:11 d2.evaluation.evaluator]: Total inference pu

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[09/15 11:11:35 d2.evaluation.evaluator]: Inference done 11/56. Dataloading: 0.0357 s/iter. Inference: 0.1602 s/iter. Eval: 0.0006 s/iter. Total: 0.1964 s/iter. ETA=0:00:08
[09/15 11:11:35 d2.evaluation.evaluator]: Inference done 11/56. Dataloading: 0.0357 s/iter. Inference: 0.1602 s/iter. Eval: 0.0006 s/iter. Total: 0.1964 s/iter. ETA=0:00:08
[09/15 11:11:40 d2.evaluation.evaluator]: Inference done 42/56. Dataloading: 0.0147 s/iter. Inference: 0.1541 s/iter. Eval: 0.0009 s/iter. Total: 0.1707 s/iter. ETA=0:00:02
[09/15 11:11:40 d2.evaluation.evaluator]: Inference done 42/56. Dataloading: 0.0147 s/iter. Inference: 0.1541 s/iter. Eval: 0.0009 s/iter. Total: 0.1707 s/iter. ETA=0:00:02
[09/15 11:11:42 d2.evaluation.evaluator]: Total inference time: 0:00:08.131220 (0.159436 s / iter per device, on 1 devices)
[09/15 11:11:42 d2.evaluation.evaluator]: Total inference time: 0:00:08.131220 (0.159436 s / iter per device, on 1 devices)
[09/15 11:11:42 d2.evaluation.evaluator]: Total inference pu

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[09/15 11:32:14 d2.evaluation.evaluator]: Inference done 11/56. Dataloading: 0.0817 s/iter. Inference: 0.1446 s/iter. Eval: 0.0044 s/iter. Total: 0.2308 s/iter. ETA=0:00:10
[09/15 11:32:14 d2.evaluation.evaluator]: Inference done 11/56. Dataloading: 0.0817 s/iter. Inference: 0.1446 s/iter. Eval: 0.0044 s/iter. Total: 0.2308 s/iter. ETA=0:00:10
[09/15 11:32:19 d2.evaluation.evaluator]: Inference done 35/56. Dataloading: 0.0259 s/iter. Inference: 0.1895 s/iter. Eval: 0.0019 s/iter. Total: 0.2182 s/iter. ETA=0:00:04
[09/15 11:32:19 d2.evaluation.evaluator]: Inference done 35/56. Dataloading: 0.0259 s/iter. Inference: 0.1895 s/iter. Eval: 0.0019 s/iter. Total: 0.2182 s/iter. ETA=0:00:04
[09/15 11:32:23 d2.evaluation.evaluator]: Total inference time: 0:00:10.563618 (0.207130 s / iter per device, on 1 devices)
[09/15 11:32:23 d2.evaluation.evaluator]: Total inference time: 0:00:10.563618 (0.207130 s / iter per device, on 1 devices)
[09/15 11:32:23 d2.evaluation.evaluator]: Total inference pu